The SPARK_HOME environment variable will be used to find the spark installation

In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/local/Cellar/apache-spark/3.2.1/libexec'

In [22]:
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [18]:
pyspark.__file__

'/usr/local/Cellar/apache-spark/3.2.1/libexec/python/pyspark/__init__.py'

Create a new spark session

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/08 00:37:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/08 00:37:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [19]:
%cd ../data

/Users/lorenz/repos/data_engineering_zoomcamp/week_5_batch_processing/data


In [20]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-03-08 00:46:53--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.73.190
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.73.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.1’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.002s  

2022-03-08 00:46:56 (6.30 MB/s) - ‘taxi+_zone_lookup.csv.1’ saved [12322/12322]



In [21]:
!head taxi+_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"


In [12]:
schema = StructType([ StructField('LocationID', IntegerType(), True), 
                      StructField('Borough', StringType(), True),
                      StructField('Zone', StringType(), True),
                      StructField('service_zone', StringType(), True)
                    ])

In [13]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('../data/taxi+_zone_lookup.csv')

In [23]:
df.schema

StructType(List(StructField(LocationID,IntegerType,true),StructField(Borough,StringType,true),StructField(Zone,StringType,true),StructField(service_zone,StringType,true)))

In [15]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [17]:
df.write.parquet('../data/zones')

In [24]:
!ls -lh

total 1469856
-rw-r--r--  1 lorenz  staff    62K Mar  7 15:59 fhvhv_head.csv
-rw-r--r--  1 lorenz  staff   717M Mar  7 15:59 fhvhv_tripdata_2021-01.csv
-rw-r--r--  1 lorenz  staff    12K Aug 17  2016 taxi+_zone_lookup.csv
drwxr-xr-x  6 lorenz  staff   192B Mar  8 00:40 zones


In [26]:
spark.stop()